# Deploying Machine Learning Models

* Train a model
* Save the model
* Use the model in a web service

![Overview](Screenshot_1.png)

**Code from previous weeks to train the model**

## Setup

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

## Read and prepare the Data

In [2]:
df = pd.read_csv("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [4]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [5]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

## Train the Model

In [6]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [7]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [8]:
C = 1.0
n_splits = 5

In [9]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.840 +- 0.008


In [10]:
scores

[0.8420689059666695,
 0.8455854357038802,
 0.8325025085289987,
 0.8301724275756219,
 0.8510401305261838]

In [11]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values

auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

## Saving and Loading the Model

* Save the model to pickle
* Load the model from pickle
* Turn our model into a Python script

### Save the model

* Use library "pickle" to write our model into a file 
* Libary to d¿save binary files

In [12]:
import pickle

In [13]:
output_file = f"model_C={C}.bin"
output_file

'model_C=1.0.bin'

In [14]:
# write a binary file
f_out = open(output_file, "wb")
# save the model and the dictionary vectorizer (we need that in order to run the model)
pickle.dump((dv, model), f_out)
# close the file
f_out.close()

In [15]:
# alternative
# this ensures that the file is closed, when the "with" statement is left
with open(output_file, "wb") as f_out:
    pickle.dump((dv, model), f_out)
    # do stuff
    
# do other stuff (file is closed)

## Load the Model

* Restart the kernel to pretend we are in a new process, that doesn't know the previous variables
* sklearn does not need to be imported, but needs to be installed

In [1]:
import pickle

In [2]:
model_file = "model_C=1.0.bin"

In [3]:
with open(model_file, "rb") as f_in:
    dv, model = pickle.load(f_in)

In [4]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [6]:
customer = {
    "gender": "female",
    "seniorcitizen": 0,
    "partner": "yes",
    "dependents": "no",
    "phoneservice": "no",
    "multiplelines": "no_phone_service",
    "internetservice": "dsl",
    "onlinesecurity": "no",
    "deviceprotection": "no",
    "techsupport": "no",
    "streamingtv": "no",
    "streamingmovies": "no",
    "contract": "month-to-month",
    "paperlessbilling": "yes",
    "paymentmethod": "electronic_check",
    "tenure": 1,
    "monthlycharges": 29.85,
    "totalcharges": 29.85
}

In [7]:
# turn this customer into a feature matrix
X = dv.transform([customer])

In [11]:
# probabilty that this customer churns
model.predict_proba(X)[0,1]

0.6638191204255028

* This is how a model can be saved and used later for prediction
* However, it is not convinient to do this all in a jupyter notebook
* Create a single .py file that does all this

## Web Services: Introduction to Flask

**What is a web service?**
* Web services method to communicate between two devices over a network
* E.g. "Google" is a web service

**How to use it:**
* Use flask to implement the web service
    * Write a python function
    * Turn it into a web service
    * Access this function from the terminal or the browser

**Example:**
* Writing a simple ping/pong app (saved in ping.py)
* Querying it with "curl" (command line utilty to communicate with web services) and browser
    * curl http://0.0.0.0:9696 or curl http://localhost:9696/ping
    * in browser: http://localhost:9696/ping

## Serving the Churn Model with Flask

* Wrapping the predict script into a Flask app
    * Cretae a web service (Churn Service) for our model
    * This should live in "/predict"
    * Information is send in JSON and response is also in JSON
        * Create JSON file for customer information
* Querying it with "requests"
    * Using the web service from the browser is this time not possible, because we used the "POST" method
* Example saved in predict_flask.py
    * For debug mode run in terminal ```python predict_flask.py``` (we set "debug=True")
    * This is only for developing, for production we need to use something else, but not plain Flask, e.g. gunicorn
    * Use Flask app for production:```gunicorn --bind 0.0.0.0:9696 predict:app ``` (in terminal)
* Preparing for production: gunicorn
* Running it on Windows with waitress (gunicorn is not supprted by windows)
    * ```waitress-serve --listen=0.0.0.0:9696 predict:app```

## Dependency and Environment Management: Pipenv

* Why we need virtual environments
    * If we have more than one application, they may depend on different versions of a package
    * The dependencies of all packages for an application must be ensured
    
![screenshot](Screenshot_2.png)
* There are different ways to use vitual environment
    * virtual env / venv
    * conda
    * pipenv -> used in this lesson
    * poetry
* Installing Pipenv
    * ```pip install pipenv```
    * ```pipenv install numpy pandas scikit-learon=0.24.2 flask```
    * ```pipenv install gunicorn```
    * This creates two new files: ```Pipfile```, ```Pipfile.lock```
    * Pipfile contains the packages we installed
    * Pipfile.lock is a JSON dictionary, that specifiees the exact version of the packeges we installed
        * When we want to work with someone together, we only need to give her the Pipenv and Pipenv.lock file, then she can do ```pipenv install``` and all dependencies will be installed as in the ```Pipfile```.
* Installing libraries with Pipenv
* Running things with Pipenv

## Environment Management: Docker

* Why we need docker
* Running a Python Image with docker
* Dockerfile
* Building a docker image
* Running a docker image

## Deploying to the Cloud: AWS Elastic Beanstalk (optional)

* Installing the eb cli
* Running eb locally
* Deploying the model

## Explore More

* Flask is not the only framework for creating web services. Try others, e.g. FastAPI
* Experiment with other ways of managing environments, e.g., virtual env, conda,  poetry
* Explore other ways of deploying web services, e.g. GCP, Azure, Heroku, Python Anywhere, etc. 

## Summary